In [2]:
def loss_function(recon_x,x,mu,logvar):# logvar  潜在的对数方差
    MSE=reconstruction_function(recon_x,x)
    KLD_element=mu.pow(2).add_(logvar.exp()).mul_(-1).add(1).add_(logvar)
    KLD=torch.sum(KLD_element).mul_(-0.5)
    #
    return MSE+KLD

    

In [1]:
import os
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader

from torchvision.datasets import mnist
from torchvision import transforms as tfs
from torchvision.utils import save_image



In [2]:
im_tfs=tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])    
        
    ])
import numpy as np
def data_tf(x):
    x=np.array(x,dtype="float32")/255
    x=(x-0.5)/0.5 # 标准化 
    x=x.reshape((-1,)) # 拉平
    x=torch.from_numpy(x)
    return x

train_set=mnist.MNIST("./data",train=True,transform=data_tf,download=False)
# test_set=mnist.MNIST("./data",train=False,transform=data_tf,download=True)
a,a_label=train_set[0]
print(a)
print(a.shape)
print(a_label)



tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.00

In [3]:
# train_set=MNIST("./mnist",transform=im_tfs,train=True, download=True)
train_data=DataLoader(train_set,batch_size=128,shuffle=True)

In [4]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()
        self.fc1=nn.Linear(784,400)
        self.fc21=nn.Linear(400,20)# mean
        self.fc22=nn.Linear(400,20) # var
        self.fc3=nn.Linear(20,400)
        self.fc4=nn.Linear(400,784)
    def encode(self,x):
        h1=F.relu(self.fc1(x))
        return self.fc21(h1),self.fc22(h1)
    def reparametrize(self,mu,logvar):
        std=logvar.mul(0.5).exp_()
        eps=torch.FloatTensor(std.size()).normal_()
        return eps.mul(std).add_(mu)
    def decode(self,z):
        h3=F.relu(self.fc3(z))
        return F.tanh(self.fc4(h3))
    def forward(self,x):
        mu,logvar=self.encode(x) # 编码
        z=self.reparametrize(mu,logvar) # 重新参数化成正太分布
        return self.decode(z),mu,logvar
    

In [5]:
net

NameError: name 'net' is not defined

In [6]:
net=VAE() # 实例化网络

x,_=train_set[0]
print(x.shape[0])
x=x.view(-1,x.shape[0])
print("x")
print(x.shape) #[784, 1]
x=Variable(x)
_,mu,var=net(x)

784
x
torch.Size([1, 784])


C:\Users\cl\Anaconda3\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [23]:
print(mu) # 网路可以输出隐藏变量的均值和方差，但是还没有训练


tensor([[-0.2107, -0.0674, -0.2846,  0.0054,  0.0567,  0.2920,  0.1648,  0.1668,
         -0.3427, -0.0207, -0.0474,  0.0107,  0.1407, -0.3136, -0.3092,  0.1778,
          0.0450,  0.3364, -0.0851,  0.0333]], grad_fn=<AddmmBackward>)


In [9]:
x=[2,4]
x.pow(2)

AttributeError: 'list' object has no attribute 'pow'

In [7]:
reconstruction_function=nn.MSELoss(size_average=False)

def loss_function(recon_x,x,mu,logvar):# logvar  潜在的对数方差
    MSE=reconstruction_function(recon_x,x)
    KLD_element=mu.pow(2).add_(logvar.exp()).mul_(-1).add(1).add_(logvar)
    KLD=torch.sum(KLD_element).mul_(-0.5)
    #
    return MSE+KLD
optimizer=torch.optim.Adam(net.parameters(),lr=1e-3)
def to_img(x):
    # 定义一个函数将最后的结果转换为图片
    x=0.5*(x+1)
    x=x.clamp(0,1)
    x=x.view(x.shape[0],1,28,28)
    return x


C:\Users\cl\Anaconda3\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
for e in range(100):
    for im,_ in train_data:
        im=im.view(im.shape[0],-1)
        im=Variable(im)
        optimizer.zero_grad()
        recon_im,mu,logvar=net(im)
        loss=loss_function(recon_im,im,mu,logvar)/im.shape[0] # 将loss 平均
        loss.backward()
        optimizer.step()
    
    if (e+1)% 20==0:
        print(str(e),loss.data)
        save=to_img(recon_im.cpu().data)
        if not os.path.exists("./vae_img"):
            os.mkdir("./vae_img")
        save_image(save,"./vae_img/image_{}.png".format(e+1))
        
        